# Relaciones semánticas con Word Embeddings
Anteriormente se ha visto cómo se puede calcular la similitud entre textos a través de la comparación e identificando tokens comunes entre ellos, pero es muy posible que debido a la complejidad del lenguaje, dos textos tengan un significado similar sin tener en común ninguna palabra.

En este apartado se introducirá el uso de word embeddings para analizar e identificar relaciones semánticas entre palabras para mejorar tareas como la clasificación de textos, la búsqueda de información o la generación de contenido.

En este caso, se explorará el uso de embeddings de palabras para analizar relaciones semánticas en textos extraídos de comentarios de usuarios de un repositorio de Github, "zigbee2mqtt". El estudio se centrará en la identificación de patrones y similutes en las discusiones sobre dispositivos IoT.

A través de modelos preentrenados y técnicas de entrenamiento personalizados, se examinará como los embeddings pueden mejorar el análisis semántico.

Se hará uso de la librería *Gensim*, ya conocida y utilizada en los apartados anteriores, para entrenar y utilizar los modelos de embeddings. Gensim proporciona una API eficiente para trabajar con embeddings como Word2Vec, FastText y GloVe, falicitando la implementación de consultas semánticas y visualización de relaciones entre términos.

## ¿Qué es ***embedding***?
Un *embedding* es una representación numérica de un objeto (palabra, frase, etc) en un espacio vectorial de menos dimensión. Se debe diferenciar entre word embedding y documento embedding. *Word embedding* es una representación vectorial de una única palabra, mientras que *document embedding* es una representación vectorial de un documento, es decir, una secuencia de palabras de cualquier longitud estrictamente mayor a 1. 

En este apartado únicamente se trabajará con **word embeddings**.

## Word Embeddings
El objetivo de un algoritmo de *embedding* es encontrar representaciones vectoriales de palabras en un espacio de dimensión *d*, de forma que palabras con significados similares tengan vectores similares. La dimensión *d* es un hiperparámetro del modelo, generalmente establecido entre 50 y 300.

Estas dimensiones no tienen un significado predefinido o interpretable, es el modelo el encargado de aprender relaciones latentes entre las palabras presentes en el texto. Cada dimensión representa una posible relación entre palabras, permitiendo que términos similares tengan valores parecidos en ciertas dimensiones.

Tras esta explicación, se podría decir que la idea básica del entrenamiento es: las palabras que aparecen en contextos similares tienen significados similares. A esto se le denomina *hipótesis distribucional* (distributional hypothesis).

## Tipos de *Embeddings*
Se han desarrollado numerosos algoritmos para entrenar word embeddings. Como en esta ocasión se utilizará la librería de Gensim nos centraremos en dos algoritmos: Word2Vec y FastText

### Word2Vec
Existen dos variantes de este algoritmo: *continuous bag-of-words model (CBOW)* y *skip-gram model*.

El modelo **CBOW** predice una palabra objetivo basándose en su contexto (las palabras que se encuentran a su alrededor). Es más eficiente en la etapa de entrenamiento porque usa promedios de los vectores de las palabras del contexto para hacer las predicciones.

La modelo **skip-gram model** predice el contexto de una palabra objetivo, es decir, trata de averiguar qué palabras rodearían a una palabra dada. Este requiere más tiempo de entrenamiento pero ofrece mejores resultados a la hora de capturar relaciones entre palabras poco frecuentes.

### GloVe
**GloVe** (global vectors) es capaz de aprender representaciones vectoriales de palabras basándose en la coocurrencia global dentro de un corpus de texto. En lugar de predecir palabras a partir de su contexto inmediato, construye una matriz que captura cuántas veces aparecen juntas las palabras y luego aplica factorización para reducir su dimensionalidad, obteniendo vectores densos donde palabras con significados similares están cerca en el espacio vectorial, permitiendo capturar mejor las relaciones semánticas y mejorar las tareas como la detección de analogías.

## Uso de *Similarity Queries* (consultas de similitud) en modelos preentrenados
Las consultas de similitud son consultas que permiten encontrar palabras, frases o documentos similares dentro de un modelo de embeddings o espacio vectorial. Se basan en métricas como la similitud del coseno, que mide qué tan cercanos se encuentran dos vectores en el espacio.

Para este primer acercamiento se utilizarán embeddings preentrenados, permitiendo ahorrar mucho tiempo.

### Pasos previos
Antes de nada, se debe cargar la configuración para la ejecución de python al igual que se ha venido haciendo en todos los apartados anteriores.

Ahora se va a hacer uso del directorio *packages* que ha sido obtenido del repositorio del libro que se está utilizando como referencia para el desarrollo del proyecto.

In [14]:
import sys, os

#Carga del archivo setup.py
%run -i ../pyenv_settings/setup.py

#Imports y configuraciones de gráficas
%run "$BASE_DIR/pyenv_settings/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

#Establece la precisión para los valores de similitud
%precision 3 
np.set_printoptions(suppress=True) #No científico para valores pequeños

#Ruta para importar los paquetes (necesario)
sys.path.append('$BASE_DIR' + '/packages')

You are working on a local system.
Files will be searched relative to "..".


### Carga del modelo preentrenado
Para cargar el modelo se utilizará la API de descarga de Gensim. Por defecto, los modelos se almacenan en la ruta *~/gensim-data*, si se desea cambiar el lugar de almacenamiento se debe modificar la ruta antes de importar la API. En este caso, yo voy a mantener la predefinida.

In [15]:
import gensim.downloader as api
import gensim

#gensim.models.utils.DEFAULT_DATADIR = os.path.join('$BASE_DIR', 'models')

Se ha hecho uso de *pandas* para mostrar los resultados en un DataFrame para visualizarlos de forma más clara en forma de lista.

In [16]:
info_df = pd.DataFrame.from_dict(api.info()['models'], orient='index')
info_df[['file_size', 'base_dataset', 'parameters']].head(5)

,file_size,base_dataset,parameters
fasttext-wiki-news-subwords-300,1005007116.00,"Wikipedia 2017, UMBC webbase corpus and statmt.org news dataset (16B tokens)",{'dimension': 300}
conceptnet-numberbatch-17-06-300,1225497562.00,"ConceptNet, word2vec, GloVe, and OpenSubtitles 2016",{'dimension': 300}
word2vec-ruscorpora-300,208427381.00,Russian National Corpus (about 250M words),"{'dimension': 300, 'window_size': 10}"
word2vec-google-news-300,1743563840.00,Google News (about 100 billion words),{'dimension': 300}
glove-wiki-gigaword-50,69182535.00,"Wikipedia 2014 + Gigaword 5 (6B tokens, uncased)",{'dimension': 50}


Se utilizará el modelo *glove-wiki-gigaword-50*. Este modelo con vectores de palabras de 50 dimensiones, y entrenado con unas 6.000 millones de palabras minúsculas, es de dimensiones reducidas pero suficiente para lo que se desea hacer en estos momentos.

El modelo se descarga del siguiente modo:

In [18]:
model = api.load('glove-wiki-gigaword-50')

El modelo pesa únicamente 66 MB. Su reducido peso se debe a que no contiene un modelo GloVe completo, únicamente los vectores de palabras. En caso de tener interés en ampliar el entrenamiento, modelos tan reducidos como este no pueden porque no se incluyen los estados internos.

### Consultas de similitud
Se puede acceder al vector de una palabra, por ejemplo *king*, por medio de la función *model.wv['king']*, o de forma aún más sencilla, con *model['king']*

In [ ]:
%precision 2 #Decimales de los valores de los resultados

v_king = model['king']
v_queen = model['queen']

print("Vector size:", model.vector_size)
print("v_king  =", v_king[:10])
print("v_queen =", v_queen[:10])

Vector size: 50
v_king  = [ 0.5   0.69 -0.6  -0.02  0.6  -0.13 -0.09  0.47 -0.62 -0.31]
v_queen = [ 0.38  1.82 -1.26 -0.1   0.36  0.6  -0.18  0.84 -0.06 -0.76]


Tras mostrar los primeros 10 elementos del vector de cada palabra, también se puente imprimir la similitud entre ambas palabras según el modelo.

In [22]:
print("similarity:", model.similarity('king', 'queen'))

similarity: 0.7839043


Como era de esperar, ambas palabras cuentan con una similitud relativamente alta.

De todos modos, se pueden comprobar cuáles son las palabras con más similitud a otra y así saber si *queen* es la palabra más similar a *king*

In [23]:
%precision 3

model.most_similar('king', topn=3)

[('prince', 0.824), ('queen', 0.784), ('ii', 0.775)]

*Prince* es la palabra más similar a *king* y *queen* es la segunda más similar.

Las puntuaciones (porcentajes) de similitud en los vectores de palabras están principalmente computadas mediante la similitud del coseno. Por ejemplo, la función *cosine_similarities* calcula la similitud entre un vector de palabra y un arrary de otros vectores de palabras.

In [24]:
v_lion = model['lion']
v_nano = model['nanotechnology']

model.cosine_similarities(v_king, [v_queen, v_lion, v_nano])

array([ 0.784,  0.478, -0.255], dtype=float32)

En base a los resultados obtenidos, se entiende que *king* es bastante similar a *queen*, hay algunas similitudes con *lion*, y no se parece en nada a *nanotechnology*. Como se ve, algunas dimensiones pueden llegar a ser negativas, pues el rango de valores de similitud va desde -1 a 1.

La función *most_similar* vista antes también acepta los parámetros *positive* y *negative*.

Resumiendo, en el parámetro *positive* se incluyen las palabras sobre las cuáles se quieren buscar las palabras más similares entre las dos, y en *negative* aquellas palabras que quieres que se eviten en las similitudes.

In [ ]:
model.most_similar(positive=['paris', 'germany'], negative=['france'], topn=3)

[('berlin', 0.920), ('frankfurt', 0.820), ('vienna', 0.818)]

En este caso por ejemplo vemos que se desean buscar las similitudes entre *paris* y *germany*, pero que no se incluyan las que se tienen con *france*. Entonces los resultados muestran: berlin, frankfurt y vienna. Se puede deducir que las similitudes se han obtenido a partir de que *paris* es la capital/ciudad más importe de Francia, y las similitudes que ha encontrado el modelo son ciudades importantes de Alemania, Berlín (capital) y Frankfurt, o capitales europeas cercanas a Alemania, como es Vienna.

In [27]:
model.most_similar(positive=['france', 'capital'], topn=1)

[('paris', 0.784)]

Si se entrenan este tipo de modelos con comentarios generados por usuarios obtenidos de redes sociales, este aprenderá relaciones entre palabras en base a estas discusiones. De este modo, se convierte en una representación de la relación que la gente cree que existe, independientemente de que exista o no.

Es un efecto secundario interesante que se tratará en el apartado a continuación.

## Entrenamiento y evaluación de un modelo de Embedding